In [38]:
import torch
import numpy as np
import json

### Dataset Description

In [49]:
diamonds = torch.arange(0,324,1)
beta_tin = torch.arange(324, 604, 1)
liquid = torch.arange(604, 673, 1)
surface_001 = torch.arange(673, 704, 1)
surface_110 = torch.arange(704, 730, 1)
surface_111 = torch.arange(730, 777, 1)
surface_111_pandey =torch.arange(777, 827, 1)
amorphous = torch.arange(827,927,1)
#First 30 is 001 surface
#31 - 56 is 110
#57 - 103 is 111
#104 - 153 is 111 pandey?

surfaces = torch.arange(673, 827)

s_surface_001 = torch.arange(0,31,1)
s_surface_110 = torch.arange(31, 57,1)
s_surface_111 = torch.arange(57,104, 1)
s_surface_111_pandey = torch.arange(104, 154,1)

### Generate indexes

In [28]:
def generate_train_test_split(n_samples):
    n_structures = n_samples
    np.random.seed(0)
    n_train = int(0.8 * n_structures)
    train_index = np.arange(n_structures)
    np.random.shuffle(train_index)
    test_index = train_index[n_train:]
    train_index = train_index[:n_train]
    
    return train_index, test_index

def generate_biased_train_test_split(n_samples):
    #Assumes 100 amorphous structures at the end
    n_structures = n_samples
    amorph_train = np.arange(n_samples-100, n_samples,1)
    np.random.seed(0)
    np.random.shuffle(amorph_train)
    
    amorph_test = amorph_train[:80]
    amorph_train = amorph_train[80:]

    n_structures = n_samples - 100
    np.random.seed(0)
    n_train = int(0.8 * n_samples)-20
    remaining_train_index = np.arange(n_structures)
    np.random.shuffle(remaining_train_index)

    remaining_test_index = remaining_train_index[n_train:]
    remaining_train_index = remaining_train_index[:n_train]

    biased_train_index = np.concatenate([remaining_train_index, amorph_train])
    biased_test_index = np.concatenate([remaining_test_index, amorph_test])
    
    return biased_train_index, biased_test_index

def generate_surface_holdout_split(n_samples):
    #Assumes that we are using the 110 surfaces for test which are located at 673 + 31st-57th index
    #26 structures
    
    n_test = int(0.2 * n_samples) - 26
    n_train = n_samples - n_test
    
    remaining_indexes = np.concatenate([np.arange(673+31), np.arange(673+57,n_samples,1)])
    indexes_110 = np.arange(673+31, 673+57,1)
    np.random.seed(0)
    
    np.random.shuffle(remaining_indexes)
    
    remaining_test_index = remaining_indexes[n_train:]
    remaining_train_index = remaining_indexes[:n_train]
    
    total_train_index = remaining_train_index
    total_test_index = np.concatenate([remaining_test_index, indexes_110])
    
    return total_train_index, total_test_index
    
def surface_holdout(n_samples):
    test_index = np.arange(31,57,1)
    train_index = np.concatenate([np.arange(31), np.arange(57, n_samples)])
    
    return train_index, test_index

n_surfaces = 154
n_bulkstructures = 773
n_total_structures = 773 + 154


surface_train_index, surface_test_index = generate_train_test_split(n_surfaces)
bulk_train_index, bulk_test_index = generate_train_test_split(n_bulkstructures)
total_train_index, total_test_index = generate_train_test_split(n_total_structures)
surface_holdout_train_index, surface_holdout_test_index = surface_holdout(n_surfaces)
bulk_biased_train_index, bulk_biased_test_index = generate_biased_train_test_split(n_bulkstructures)
total_biased_train_index, total_biased_test_index = generate_biased_train_test_split(n_total_structures)
holdout_train_index, holdout_test_index = generate_surface_holdout_split(n_total_structures)

In [33]:
def reverse_index(train_align, test_align, train_index, test_index):
    total_number = len(train_align) + len(test_align)
    original_index = torch.zeros(total_number).float()
    
    original_index.index_add_(0, train_index, train_align)
    original_index.index_add_(0, test_index, test_align)
    
    return original_index

In [40]:
f = open("./vacuum_reference.json")
vacuum_reference = json.load(f)
vacuum_reference = torch.tensor([vacuum_reference[i] for i in vacuum_reference])

deepest_core_reference = np.load("./deepest_core_reference.npy")

In [42]:
adam_corr_train_shifts = torch.load("./adam_corr_train_shifts.pt")
adam_corr_test_shifts = torch.load("./adam_corr_test_shifts.pt")

In [45]:
adam_corr_shifts = reverse_index(adam_corr_train_shifts.float(), adam_corr_test_shifts.float(), torch.tensor(total_train_index), torch.tensor(total_test_index))

In [94]:
torch.mean(torch.abs((adam_corr_shifts[surfaces] * 0.05 + 6.4462) - vacuum_reference))

tensor(0.2044)

In [96]:
def functiona():
    print (a)

In [ ]:
0.2eV error

In [71]:
vacuum_reference

tensor([-0.1269, -0.1822, -0.1325, -0.1535, -0.1991, -0.2056, -0.2090, -0.2104,
        -0.2143, -0.2171, -0.2107, -0.2115, -0.2010, -0.2047, -0.2085, -0.2044,
        -0.2131, -0.2056, -0.2052, -0.2096, -0.2083, -0.2119, -0.2199, -0.2117,
        -0.2149, -0.2511, -0.2018, -0.1975, -0.2372, -0.1941, -0.2144, -0.1968,
        -0.2082, -0.2033, -0.2060, -0.1981, -0.2007, -0.1994, -0.2090, -0.1971,
        -0.1988, -0.2072, -0.2086, -0.2011, -0.2024, -0.2242, -0.2116, -0.2176,
        -0.2151, -0.1897, -0.2043, -0.2031, -0.2024, -0.2066, -0.2074, -0.2128,
        -0.2189, -0.1342, -0.1165, -0.1383, -0.1379, -0.1098, -0.1282, -0.1133,
        -0.1205, -0.1504, -0.1323, -0.1192, -0.1226, -0.1309, -0.1177, -0.1281,
        -0.1374, -0.1237, -0.0989, -0.1141, -0.1313, -0.1023, -0.1323, -0.1167,
        -0.1553, -0.1264, -0.1040, -0.1226, -0.1369, -0.0913, -0.0966, -0.1046,
        -0.1075, -0.1084, -0.1113, -0.1159, -0.1218, -0.1199, -0.1167, -0.1158,
        -0.1154, -0.1105, -0.1128, -0.11

In [69]:
torch.mean(adam_corr_shifts[surfaces] * 0.05 - vacuum_reference)

tensor(-6.4462)

In [59]:
vacuum_reference

tensor([-0.1269, -0.1822, -0.1325, -0.1535, -0.1991, -0.2056, -0.2090, -0.2104,
        -0.2143, -0.2171, -0.2107, -0.2115, -0.2010, -0.2047, -0.2085, -0.2044,
        -0.2131, -0.2056, -0.2052, -0.2096, -0.2083, -0.2119, -0.2199, -0.2117,
        -0.2149, -0.2511, -0.2018, -0.1975, -0.2372, -0.1941, -0.2144, -0.1968,
        -0.2082, -0.2033, -0.2060, -0.1981, -0.2007, -0.1994, -0.2090, -0.1971,
        -0.1988, -0.2072, -0.2086, -0.2011, -0.2024, -0.2242, -0.2116, -0.2176,
        -0.2151, -0.1897, -0.2043, -0.2031, -0.2024, -0.2066, -0.2074, -0.2128,
        -0.2189, -0.1342, -0.1165, -0.1383, -0.1379, -0.1098, -0.1282, -0.1133,
        -0.1205, -0.1504, -0.1323, -0.1192, -0.1226, -0.1309, -0.1177, -0.1281,
        -0.1374, -0.1237, -0.0989, -0.1141, -0.1313, -0.1023, -0.1323, -0.1167,
        -0.1553, -0.1264, -0.1040, -0.1226, -0.1369, -0.0913, -0.0966, -0.1046,
        -0.1075, -0.1084, -0.1113, -0.1159, -0.1218, -0.1199, -0.1167, -0.1158,
        -0.1154, -0.1105, -0.1128, -0.11

In [57]:
torch.min(adam_corr_train_shifts, 0)

torch.return_types.min(
values=tensor(-142),
indices=tensor(454))

In [63]:
deepest_core_reference

array([ 0.72565923,  0.69334156,  0.72267476,  0.65729607,  0.69038358,
        0.67575365,  0.67789478,  0.74287965,  0.73520293,  0.66940705,
        0.75388326,  0.67962418,  0.75309393,  0.67809806,  0.66200593,
        0.68852234,  0.6980107 ,  0.69515279,  0.71697716,  0.71676054,
        0.77145731,  0.72849885,  0.63435355,  0.72381027,  0.74874937,
        0.74924887,  0.78512607,  0.71740105,  0.72929014,  0.78190546,
        0.77345609,  0.73414133,  0.72135374,  0.78483447,  0.73587872,
        0.70432459,  0.72593537,  0.73649574,  0.74462285,  0.75133543,
        0.73007797,  0.75246892,  0.79227774,  0.77631127,  0.73351916,
        0.68640386,  0.61568481,  0.68357789,  0.79958783,  0.8035231 ,
        0.77965418,  0.80257177,  0.78901694,  0.74870729,  0.79656902,
        0.79936051,  0.80657475,  0.79319723,  0.74400754,  0.7621769 ,
        0.79708714,  0.73504051,  0.6654097 ,  0.75881337,  0.70194451,
        0.66850316,  0.777604  ,  0.74172273,  0.69273589,  0.78

tensor([-139., -139., -139., -139., -139., -139., -139., -139., -139., -139.,
        -139., -139., -139., -139., -139., -139., -139., -139., -139., -139.,
        -139., -139., -139., -139., -139., -139., -139., -139., -139., -139.,
        -139., -138., -138., -138., -138., -138., -138., -138., -138., -138.,
        -138., -138., -138., -138., -138., -138., -138., -138., -138., -138.,
        -138., -138., -138., -138., -138., -138., -138., -138., -138., -138.,
        -138., -138., -138., -138., -138., -138., -138., -138., -138., -138.,
        -138., -138., -138., -138., -138., -138., -138., -138., -138., -138.,
        -138., -138., -138., -138., -138., -138., -138., -138., -138., -138.,
        -138., -137., -137., -137., -137., -137., -137., -137., -137., -137.,
        -137., -137., -137., -137., -137., -137., -137., -137., -137., -137.,
        -137., -137., -137., -137., -137., -137., -137., -137., -137., -137.,
        -137., -137., -137., -137., -137., -137., -137., -137., 